# OAS->requests

In [1]:
#| default_exp oas_to_requests

In [6]:
#| hide
from nbdev.showdoc import *

In [10]:
#| export
import requests
# from openapi_schema_pydantic import OpenAPI
from pprint import pprint
import json
import yaml

## OAS inspection

In [1]:
!wget https://petstore3.swagger.io/api/v3/openapi.json

--2024-11-06 14:33:32--  https://petstore3.swagger.io/api/v3/openapi.json
Resolving petstore3.swagger.io (petstore3.swagger.io)... 3.232.125.100, 18.215.136.63
Connecting to petstore3.swagger.io (petstore3.swagger.io)|3.232.125.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘openapi.json’

openapi.json            [ <=>                ]  15.71K  --.-KB/s    in 0.1s    

2024-11-06 14:33:32 (141 KB/s) - ‘openapi.json’ saved [16090]



In [24]:
#| export
BASE_URL = "https://petstore3.swagger.io/api/v3"

# Load and parse the OAS file
with open("openapi.json") as f:
    spec_dict = json.load(f)
# oas = OpenAPI.parse_obj(spec_dict)
oas = spec_dict
oas

{'openapi': '3.0.2',
 'info': {'title': 'Swagger Petstore - OpenAPI 3.0',
  'description': "This is a sample Pet Store Server based on the OpenAPI 3.0 specification.  You can find out more about\nSwagger at [http://swagger.io](http://swagger.io). In the third iteration of the pet store, we've switched to the design first approach!\nYou can now help us improve the API whether it's by making changes to the definition itself or to the code.\nThat way, with time, we can improve the API in general, and expose some of the new features in OAS3.\n\nSome useful links:\n- [The Pet Store repository](https://github.com/swagger-api/swagger-petstore)\n- [The source API definition for the Pet Store](https://github.com/swagger-api/swagger-petstore/blob/master/src/main/resources/openapi.yaml)",
  'termsOfService': 'http://swagger.io/terms/',
  'contact': {'email': 'apiteam@swagger.io'},
  'license': {'name': 'Apache 2.0',
   'url': 'http://www.apache.org/licenses/LICENSE-2.0.html'},
  'version': '1.0.1

In [27]:
#| export
def resolve_refs(
    oas: dict,  # OpenAPI spec as a dictionary
) -> dict:  # OpenAPI spec as a dictionary with resolved references
    """
    Resolve a reference in an OpenAPI spec.
    """
    oas_flatten = oas.copy()
    # Store resolved references
    refs = {}

    # Traverse the components section of the spec
    for section, items in oas_flatten["components"].items():
        for item in items:
            refs[f"#/components/{section}/{item}"] = oas_flatten["components"][section][item]

    # Traverse the spec object to resolve references in the paths
    def traverse_attach(obj):
        if isinstance(obj, dict):  
            # If the object is a reference, resolve it
            if "$ref" in obj:
                resolved_obj = refs[obj["$ref"]]
                return traverse_attach(resolved_obj)
            
            for key, value in obj.items():
                # Traverse the value if it is not a reference
                obj[key] = traverse_attach(value)

        elif isinstance(obj, list):
            # Traverse each item in the list
            return [traverse_attach(item) for item in obj]
        
        return obj
    
    # Traverse the spec object to resolve references in the paths
    traverse_attach(oas_flatten)

    return oas_flatten

In [28]:
show_doc(resolve_refs)

---

### resolve_refs

>      resolve_refs (oas:dict)

*Resolve a reference in an OpenAPI spec.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| oas | dict | OpenAPI spec as a dictionary |
| **Returns** | **dict** | **OpenAPI spec as a dictionary with resolved references** |

In [31]:
oas_flatten = resolve_refs(oas)
oas_flatten

{'openapi': '3.0.2',
 'info': {'title': 'Swagger Petstore - OpenAPI 3.0',
  'description': "This is a sample Pet Store Server based on the OpenAPI 3.0 specification.  You can find out more about\nSwagger at [http://swagger.io](http://swagger.io). In the third iteration of the pet store, we've switched to the design first approach!\nYou can now help us improve the API whether it's by making changes to the definition itself or to the code.\nThat way, with time, we can improve the API in general, and expose some of the new features in OAS3.\n\nSome useful links:\n- [The Pet Store repository](https://github.com/swagger-api/swagger-petstore)\n- [The source API definition for the Pet Store](https://github.com/swagger-api/swagger-petstore/blob/master/src/main/resources/openapi.yaml)",
  'termsOfService': 'http://swagger.io/terms/',
  'contact': {'email': 'apiteam@swagger.io'},
  'license': {'name': 'Apache 2.0',
   'url': 'http://www.apache.org/licenses/LICENSE-2.0.html'},
  'version': '1.0.1

In [6]:
#| export
def get_request_body_from_schema(operation, oas):
    """Generates a default request body from the schema if `data` is not provided."""
    requestBody = operation.get("requestBody", {})
    if not requestBody: return None
    content = requestBody.get("content", {})
    schema_ref = content.get("application/json", {}).get("schema", {}).get("$ref")
    if schema_ref:
        schema_name = schema_ref.split("/")[-1]
        schema = oas.components.schemas.get(schema_name)
        if schema:
            return {"title": "Example Task", "completed": False}  # Example structure
    return None

In [7]:
#| export
def __generate_request_by_operation_id(oas, operation_id, parameters=None, data=None):
    # Search for the operation by `operationId`
    for path, path_item in oas.paths.items():
        for method, operation in path_item.dict().items():
            if operation and operation.get("operationId") == operation_id:
                url = f"{BASE_URL}{path}"
                headers = {"Content-Type": "application/json"}
                
                # Determine request body: use `data` if provided; otherwise, generate from schema
                request_body = data or get_request_body_from_schema(operation, oas)

                # Make the HTTP request dynamically
                response = requests.request(method.upper(), url, params=parameters, json=request_body, headers=headers)
                print(url, request_body)
                return response.json() if response.status_code in (200, 201) else response.status_code
    print(f"OperationId '{operation_id}' not found.")
    return None

In [11]:
#| export
def generate_request_by_operation_id(operation_id, parameters=None, data=None):
    return __generate_request_by_operation_id(oas, operation_id, parameters, data)

In [9]:
ans = generate_request_by_operation_id("findPetsByStatus", parameters={"status":"sold"})
ans[:3]

https://petstore3.swagger.io/api/v3/pet/findByStatus None


[{'id': 5081695936906578352,
  'category': {'id': 411886826470505861, 'name': ''},
  'name': "I4(u&blLw7Y~Xq:_A*6vU=;H]5R~#{BW-2[[#aI/q!'h*Y8jPg?NINmd+r",
  'photoUrls': ["neQ~Md/th'BL$*9OP.Ctd[~m68z`As{1Cf}e#T=Hz^*?v|sfAFat",
   '*v29XDy|TQ'],
  'tags': [{'id': 4695381958756277942,
    'name': "5Hb3X1h]ngm9'b!a^&!/T^DPM&6:D(c*F#d`%Fn3IVW!jkF5HI{d0qQj"},
   {'id': 3829192767566662587,
    'name': "!Bxsu/-ezDuPK/l'd])!Lepp4Kk:hjhd::/j~%T/y(;MmHFye8:MOs2[,4`"},
   {'id': 7415501824273923801, 'name': '%Ky97j'}],
  'status': 'sold'},
 {'id': 4987019593767351499,
  'category': {'id': 411886826470505861, 'name': ''},
  'name': "I4(u&blLw7Y~Xq:_A*6vU=;H]5R~#{BW-2[[#aI/q!'h*Y8jPg?NINmd+r",
  'photoUrls': ["neQ~Md/th'BL$*9OP.Ctd[~m68z`As{1Cf}e#T=Hz^*?v|sfAFat",
   '*v29XDy|TQ'],
  'tags': [{'id': 4695381958756277942,
    'name': "5Hb3X1h]ngm9'b!a^&!/T^DPM&6:D(c*F#d`%Fn3IVW!jkF5HI{d0qQj"},
   {'id': 3829192767566662587,
    'name': "!Bxsu/-ezDuPK/l'd])!Lepp4Kk:hjhd::/j~%T/y(;MmHFye8:MOs2[,4`"},


In [10]:
#| hide
import nbdev; nbdev.nbdev_export()